In [1]:
import pandas as pd
import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, BatchNormalization,Input
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import np_utils
import keras.backend as Kr
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import class_weight
import numpy as np
from numpy import exp
# Library for Gaussian process
# import GPy
##Library for visualization
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib;matplotlib.rcParams['figure.figsize'] = (10,7)
import pylab 
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import GPy

Using TensorFlow backend.


In [2]:
df_loc = pd.read_csv("2D_biv_matern_8100_projection.csv", sep = ",")
df_loc.head()

,x,y,var1,var2,proj_var1,proj_var2,threshold,dist_frm_orig,threshold2,constant,class
0,0.898876,0.0,-2.916991,-4.270493,-2.842809,-4.380952,1,11.726556,3,0,3
1,0.910112,0.0,-3.123750,-4.581233,-3.129122,-4.573235,1,14.207576,3,0,3
2,0.921348,0.0,-2.942157,-4.591575,-3.008760,-4.492402,1,13.135607,3,0,3
3,0.932584,0.0,-3.018496,-4.650880,-3.088819,-4.546168,1,13.843947,3,0,3
4,0.943820,0.0,-3.403380,-4.727132,-3.389361,-4.748008,1,16.669045,4,0,4


In [8]:
df_train,df_test = train_test_split(df_loc, test_size = 0.06)

## Projected data classification 

### 140 classes 

In [9]:
# model_list_var1 = list()
# df_train1 = df_train
# print(df_train1.head(1))
# df_loc["threshold_var1"] = df_loc["threshold_var1"] - 1
# dummy_y = np_utils.to_categorical(df_loc["threshold_var1"])
# n = dummy_y.shape[1]
# print(n)

In [11]:
len(np.unique(df_train["class"]))

140

In [13]:
model_list_var1 = list()
df_train1 = df_train.copy()
# print(df_train1.head(1))
df_train1["class"] = df_train1["class"] - 1
dummy_y = np_utils.to_categorical(df_train1["class"])
n = dummy_y.shape[1]
print(n)
N = len(df_train1)
print(N)
s = np.vstack((df_train1["x"],df_train1["y"])).T

num_basis = [5**2,7**2,11**2]
knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
##Wendland kernel
K = 0
phi = np.zeros((N, sum(num_basis)))

for res in range(len(num_basis)):
    theta = 1/np.sqrt(num_basis[res])*2.5
    knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
    knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(s-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi[j,i + K] = 0
    K = K + num_basis[res]
def model_function():
    class_weights = class_weight.compute_class_weight('balanced',np.unique(df_train["class"]),
                                             df_train["class"])
    class_weight_dict = dict(enumerate(class_weights))
    # DeepKriging model for continuous data
    model = Sequential()
    # model.add(Dense(100, input_dim = 2,  kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(100, input_dim = phi.shape[1],  
            kernel_initializer='he_uniform', activation='relu'))
    # model.add(Dropout(rate=0.5))
    # model.add(BatchNormalization())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    #     model.add(Dense(100, activation='relu'))
    # model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    #     model.add(Dense(50, activation='relu'))
    #     model.add(Dense(50, activation='relu'))
    #     model.add(Dense(10, activation='relu'))
    #model.add(Dense(50, activation='relu'))
    #model.add(Dropout(rate=0.5))
    #     model.add(Dense(10, activation='relu'))
    #model.add(BatchNormalization())
    model.add(Dense(10, activation='relu'))
    model.add(Dense(n, activation='softmax'))
    NB_START_EPOCHS = 50 
    # NB_START_EPOCHS = 200  # Number of epochs we usually start to train with
    BATCH_SIZE = 64  
    fold_no = 1
    optimizer = keras.optimizers.Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    callbacks = [EarlyStopping(monitor='val_accuracy', patience=200),
                 ModelCheckpoint(filepath='indicator_kriging.h5', 
                                 monitor='val_accuracy', save_best_only=True)]
    print("running model for variable 1.")
    result = model.fit(phi, dummy_y, callbacks=callbacks, class_weight = class_weight_dict,
               validation_split = 0.1, epochs = 500, batch_size = 128, verbose = 2)

    model = keras.models.load_model('indicator_kriging.h5')
    return model
model_list_var1.append(model_function())

140
7614


 /Users/nagp/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning:Pass classes=[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140], y=3940     64
4096     69
5024     81
6186    103
1453     25
       ... 
1828     28
4277     65
3624     64
7979    127
47        4
Name: class, Length: 7614, dtype: int64 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error


running model for variable 1.
Train on 6852 samples, validate on 762 samples
Epoch 1/500
 - 1s - loss: 4.9204 - accuracy: 0.0105 - val_loss: 5.1263 - val_accuracy: 0.0039
Epoch 2/500
 - 0s - loss: 4.8615 - accuracy: 0.0149 - val_loss: 5.1100 - val_accuracy: 0.0066
Epoch 3/500
 - 0s - loss: 4.4704 - accuracy: 0.0344 - val_loss: 4.8218 - val_accuracy: 0.0394
Epoch 4/500
 - 0s - loss: 4.0224 - accuracy: 0.0660 - val_loss: 4.5767 - val_accuracy: 0.0564
Epoch 5/500
 - 0s - loss: 3.6450 - accuracy: 0.0997 - val_loss: 4.5958 - val_accuracy: 0.0997
Epoch 6/500
 - 0s - loss: 3.3678 - accuracy: 0.1299 - val_loss: 4.9377 - val_accuracy: 0.0906
Epoch 7/500
 - 0s - loss: 3.1837 - accuracy: 0.1421 - val_loss: 5.1902 - val_accuracy: 0.1115
Epoch 8/500
 - 0s - loss: 3.0346 - accuracy: 0.1547 - val_loss: 5.1803 - val_accuracy: 0.1588
Epoch 9/500
 - 1s - loss: 2.8897 - accuracy: 0.1823 - val_loss: 5.2126 - val_accuracy: 0.1732
Epoch 10/500
 - 0s - loss: 2.7702 - accuracy: 0.1839 - val_loss: 5.7238 - val

Epoch 87/500
 - 1s - loss: 1.6208 - accuracy: 0.3933 - val_loss: 7.7695 - val_accuracy: 0.3425
Epoch 88/500
 - 0s - loss: 1.5941 - accuracy: 0.4003 - val_loss: 7.7979 - val_accuracy: 0.3517
Epoch 89/500
 - 1s - loss: 1.6745 - accuracy: 0.3974 - val_loss: 7.7338 - val_accuracy: 0.3543
Epoch 90/500
 - 1s - loss: 1.5564 - accuracy: 0.4151 - val_loss: 7.7964 - val_accuracy: 0.3281
Epoch 91/500
 - 1s - loss: 1.5883 - accuracy: 0.4062 - val_loss: 7.8764 - val_accuracy: 0.3307
Epoch 92/500
 - 1s - loss: 1.6797 - accuracy: 0.3835 - val_loss: 7.7425 - val_accuracy: 0.3438
Epoch 93/500
 - 1s - loss: 1.6055 - accuracy: 0.4053 - val_loss: 7.9796 - val_accuracy: 0.3635
Epoch 94/500
 - 1s - loss: 1.5368 - accuracy: 0.4219 - val_loss: 7.8154 - val_accuracy: 0.3583
Epoch 95/500
 - 1s - loss: 1.5787 - accuracy: 0.4133 - val_loss: 8.3615 - val_accuracy: 0.3176
Epoch 96/500
 - 1s - loss: 1.5782 - accuracy: 0.4048 - val_loss: 7.6810 - val_accuracy: 0.3766
Epoch 97/500
 - 1s - loss: 1.5498 - accuracy: 0.42

Epoch 173/500
 - 1s - loss: 1.4275 - accuracy: 0.4450 - val_loss: 8.8607 - val_accuracy: 0.3753
Epoch 174/500
 - 1s - loss: 1.3824 - accuracy: 0.4653 - val_loss: 8.7089 - val_accuracy: 0.3898
Epoch 175/500
 - 1s - loss: 1.3625 - accuracy: 0.4758 - val_loss: 8.5418 - val_accuracy: 0.3832
Epoch 176/500
 - 1s - loss: 1.3451 - accuracy: 0.4745 - val_loss: 8.6606 - val_accuracy: 0.3688
Epoch 177/500
 - 1s - loss: 1.3515 - accuracy: 0.4619 - val_loss: 8.9939 - val_accuracy: 0.3832
Epoch 178/500
 - 1s - loss: 1.2798 - accuracy: 0.4877 - val_loss: 9.0181 - val_accuracy: 0.3780
Epoch 179/500
 - 1s - loss: 1.3420 - accuracy: 0.4803 - val_loss: 8.9021 - val_accuracy: 0.3727
Epoch 180/500
 - 1s - loss: 1.3738 - accuracy: 0.4599 - val_loss: 8.6860 - val_accuracy: 0.4042
Epoch 181/500
 - 1s - loss: 1.3124 - accuracy: 0.4796 - val_loss: 8.8956 - val_accuracy: 0.3661
Epoch 182/500
 - 1s - loss: 1.4788 - accuracy: 0.4390 - val_loss: 8.3954 - val_accuracy: 0.3675
Epoch 183/500
 - 1s - loss: 1.3282 - acc

Epoch 259/500
 - 1s - loss: 1.1709 - accuracy: 0.5190 - val_loss: 9.8595 - val_accuracy: 0.4029
Epoch 260/500
 - 1s - loss: 1.1346 - accuracy: 0.5298 - val_loss: 10.1996 - val_accuracy: 0.3596
Epoch 261/500
 - 1s - loss: 1.1836 - accuracy: 0.4962 - val_loss: 9.7006 - val_accuracy: 0.4108
Epoch 262/500
 - 0s - loss: 1.2184 - accuracy: 0.5038 - val_loss: 9.7841 - val_accuracy: 0.3845
Epoch 263/500
 - 0s - loss: 1.1762 - accuracy: 0.5105 - val_loss: 9.5011 - val_accuracy: 0.4108
Epoch 264/500
 - 0s - loss: 1.1639 - accuracy: 0.5152 - val_loss: 10.0218 - val_accuracy: 0.3950
Epoch 265/500
 - 0s - loss: 1.1774 - accuracy: 0.5058 - val_loss: 10.2466 - val_accuracy: 0.4147
Epoch 266/500
 - 0s - loss: 1.1709 - accuracy: 0.5134 - val_loss: 10.5688 - val_accuracy: 0.3950
Epoch 267/500
 - 0s - loss: 1.2174 - accuracy: 0.5015 - val_loss: 10.8105 - val_accuracy: 0.3819
Epoch 268/500
 - 0s - loss: 1.1794 - accuracy: 0.5112 - val_loss: 9.7725 - val_accuracy: 0.3898
Epoch 269/500
 - 0s - loss: 1.1890 

Epoch 344/500
 - 0s - loss: 1.1132 - accuracy: 0.5290 - val_loss: 11.0634 - val_accuracy: 0.4016
Epoch 345/500
 - 0s - loss: 1.0597 - accuracy: 0.5550 - val_loss: 10.4474 - val_accuracy: 0.3740
Epoch 346/500
 - 0s - loss: 1.0591 - accuracy: 0.5506 - val_loss: 10.8521 - val_accuracy: 0.3950
Epoch 347/500
 - 0s - loss: 1.0676 - accuracy: 0.5436 - val_loss: 11.1695 - val_accuracy: 0.4042
Epoch 348/500
 - 0s - loss: 1.0884 - accuracy: 0.5358 - val_loss: 10.8895 - val_accuracy: 0.3793
Epoch 349/500
 - 1s - loss: 1.1593 - accuracy: 0.5306 - val_loss: 10.8042 - val_accuracy: 0.3924
Epoch 350/500
 - 0s - loss: 1.0732 - accuracy: 0.5450 - val_loss: 10.2004 - val_accuracy: 0.4003
Epoch 351/500
 - 1s - loss: 1.0547 - accuracy: 0.5425 - val_loss: 10.6218 - val_accuracy: 0.4081
Epoch 352/500
 - 1s - loss: 1.0540 - accuracy: 0.5414 - val_loss: 11.0869 - val_accuracy: 0.3714
Epoch 353/500
 - 1s - loss: 1.0433 - accuracy: 0.5474 - val_loss: 10.0424 - val_accuracy: 0.4094
Epoch 354/500
 - 1s - loss: 1.

Epoch 429/500
 - 1s - loss: 1.0468 - accuracy: 0.5617 - val_loss: 11.4739 - val_accuracy: 0.3911
Epoch 430/500
 - 1s - loss: 1.0306 - accuracy: 0.5610 - val_loss: 11.6023 - val_accuracy: 0.3622
Epoch 431/500
 - 1s - loss: 0.9914 - accuracy: 0.5779 - val_loss: 11.2726 - val_accuracy: 0.3950
Epoch 432/500
 - 1s - loss: 0.9725 - accuracy: 0.5684 - val_loss: 11.6706 - val_accuracy: 0.3911
Epoch 433/500
 - 0s - loss: 0.9857 - accuracy: 0.5654 - val_loss: 11.2165 - val_accuracy: 0.4134
Epoch 434/500
 - 0s - loss: 0.9319 - accuracy: 0.5766 - val_loss: 11.4454 - val_accuracy: 0.4029
Epoch 435/500
 - 0s - loss: 0.9578 - accuracy: 0.5695 - val_loss: 11.8096 - val_accuracy: 0.4147
Epoch 436/500
 - 0s - loss: 1.0116 - accuracy: 0.5714 - val_loss: 11.2732 - val_accuracy: 0.3596
Epoch 437/500
 - 0s - loss: 0.9621 - accuracy: 0.5679 - val_loss: 11.4922 - val_accuracy: 0.4081
Epoch 438/500
 - 0s - loss: 0.9504 - accuracy: 0.5741 - val_loss: 11.7692 - val_accuracy: 0.3845
Epoch 439/500
 - 0s - loss: 0.

In [14]:

N = len(df_loc)
s = np.vstack((df_loc["x"],df_loc["y"])).T

num_basis = [5**2,7**2,11**2]
knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
##Wendland kernel
K = 0
phi = np.zeros((N, sum(num_basis)))

for res in range(len(num_basis)):
    theta = 1/np.sqrt(num_basis[res])*2.5
    knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
    knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(s-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi[j,i + K] = 0
    K = K + num_basis[res]
model = model_list_var1[0]
pred = model.predict(phi)
pred_df = pd.DataFrame(pred)

In [15]:
pred_df

,0,1,2,3,4,5,6,7,8,9,...,130,131,132,133,134,135,136,137,138,139
0,1.739566e-19,0.003042,0.324946,0.061119,0.000007,4.717139e-10,1.525096e-15,1.553062e-26,1.268484e-38,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,9.664692e-20,0.000237,0.677809,0.192017,0.000538,4.095351e-08,1.709391e-13,6.433325e-23,4.303630e-34,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,1.026823e-18,0.000452,0.680863,0.299867,0.001125,1.767172e-07,1.812267e-12,1.689698e-21,1.632681e-32,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,5.360958e-17,0.000624,0.410871,0.582680,0.005241,3.654676e-06,2.297093e-10,1.952453e-18,8.361681e-29,2.024800e-34,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,1.020634e-14,0.000477,0.110420,0.861035,0.027893,1.640463e-04,1.390596e-07,3.388932e-14,1.699682e-23,1.642315e-27,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8095,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,4.836693e-09,5.379264e-03,4.576569e-07,0.836334,0.120163,0.031579,8.460199e-12,4.232425e-22,2.840048e-25,0.000000e+00
8096,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.685192e-34,1.758181e-11,7.126846e-12,0.011822,0.209085,0.764630,8.256384e-04,1.136447e-08,9.460909e-10,3.536104e-26
8097,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.605578e-30,5.830494e-10,4.743375e-11,0.038953,0.291365,0.662441,3.119585e-05,2.609468e-11,7.517369e-13,3.780268e-31
8098,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.188502e-20,4.146426e-07,2.318453e-11,0.484962,0.230948,0.186371,5.050597e-11,2.660673e-21,1.734875e-23,0.000000e+00


In [16]:
pred_df.to_csv("projection_probs_deep.csv")

### Modelling Variable 2

In [37]:
model_list_var2 = list()
df_train1 = df_train
print(df_train1.head(1))
df_train1["threshold_var2"] = df_train1["threshold_var2"] - 1
dummy_y = np_utils.to_categorical(df_train1["threshold_var2"])
n = dummy_y.shape[1]
print(n)
N = len(df_train1)
print(N)
s = np.vstack((df_train1["x"],df_train1["y"])).T

num_basis = [2**2,5**2,9**2]
knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
##Wendland kernel
K = 0
phi = np.zeros((N, sum(num_basis)))

for res in range(len(num_basis)):
    theta = 1/np.sqrt(num_basis[res])*2.5
    knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
    knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(s-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi[j,i + K] = 0
    K = K + num_basis[res]
def model_function():
    class_weights = class_weight.compute_class_weight('balanced',np.unique(df_train1["threshold_var1"]),
                                             df_train1["threshold_var1"])
    class_weight_dict = dict(enumerate(class_weights))
    # DeepKriging model for continuous data
    model = Sequential()
    # model.add(Dense(100, input_dim = 2,  kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(100, input_dim = phi.shape[1],  
            kernel_initializer='he_uniform', activation='relu'))
    # model.add(Dropout(rate=0.5))
    # model.add(BatchNormalization())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    #     model.add(Dense(100, activation='relu'))
    # model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    #     model.add(Dense(50, activation='relu'))
    #     model.add(Dense(50, activation='relu'))
    #     model.add(Dense(10, activation='relu'))
    #model.add(Dense(50, activation='relu'))
    #model.add(Dropout(rate=0.5))
    #     model.add(Dense(10, activation='relu'))
    #model.add(BatchNormalization())
    model.add(Dense(10, activation='relu'))
    model.add(Dense(n, activation='softmax'))
    NB_START_EPOCHS = 50 
    # NB_START_EPOCHS = 200  # Number of epochs we usually start to train with
    BATCH_SIZE = 64  
    fold_no = 1
    optimizer = keras.optimizers.Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    callbacks = [EarlyStopping(monitor='val_accuracy', patience=200),
                 ModelCheckpoint(filepath='indicator_kriging.h5', 
                                 monitor='val_accuracy', save_best_only=True)]
    print("running model for variable 1.")
    result = model.fit(phi, dummy_y, callbacks=callbacks, class_weight = class_weight_dict,
               validation_split = 0.1, epochs = 500, batch_size = 64, verbose = 0)

    model = keras.models.load_model('indicator_kriging.h5')
    return model
model_list_var1.append(model_function())

             x         y      var1      var2  proj_var1  proj_var2  threshold  \
4560  0.157303  0.988764 -5.350081 -5.250944  -5.306286  -5.300444          4   

      dist_frm_orig  threshold2  threshold_var1  threshold_var2  
4560      50.197071          10              15              16  
40
7290


 /Users/nagp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 /Users/nagp/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning:Pass classes=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39], y=4560    15
1989    18
2875    20
509     26
2565    25
        ..
1533    26
6824    10
7172    29
4305    19
4022    20
Name: threshold_var1, Length: 7290, dtype: int64 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error


running model for variable 1.


In [39]:
N = len(df_test)
s = np.vstack((df_test["x"],df_test["y"])).T

num_basis = [2**2,5**2,9**2]
knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
##Wendland kernel
K = 0
phi = np.zeros((N, sum(num_basis)))

for res in range(len(num_basis)):
    theta = 1/np.sqrt(num_basis[res])*2.5
    knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
    knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(s-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi[j,i + K] = 0
    K = K + num_basis[res]
model = model_list_var1[1]
pred = model.predict(phi)
pred_df = pd.DataFrame(pred)

In [40]:
pred_df.to_csv("var2_probs.csv")

## Modelling the bivariate variable

In [51]:
model_list = list()

In [52]:
for h in range(7):
    df_train1 = df_train.loc[df_train['threshold'] == h+1]
    if(len(np.unique(df_train1['threshold2'])) != 20):
        df_train1 = df_loc.loc[df_loc['threshold'] == h+1]
    else:
        pass
    print(df_train1.head(1))
    df_train1["threshold2"] = df_train1["threshold2"] - 1
    dummy_y = np_utils.to_categorical(df_train1["threshold2"])
    n = dummy_y.shape[1]
    N = len(df_train1)
    print(n)
    print(N)
    s = np.vstack((df_train1["x"],df_train1["y"])).T

    num_basis = [2**2,5**2,9**2]
    knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
    ##Wendland kernel
    K = 0
    phi = np.zeros((N, sum(num_basis)))

    for res in range(len(num_basis)):
        theta = 1/np.sqrt(num_basis[res])*2.5
        knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
        knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
        for i in range(num_basis[res]):
            d = np.linalg.norm(s-knots[i,:],axis=1)/theta
            for j in range(len(d)):
                if d[j] >= 0 and d[j] <= 1:
                    phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
                else:
                    phi[j,i + K] = 0
        K = K + num_basis[res]
    def model_function():
        class_weights = class_weight.compute_class_weight('balanced',np.unique(df_train1["threshold2"]),
                                                 df_train1["threshold2"])
        class_weight_dict = dict(enumerate(class_weights))
        # DeepKriging model for continuous data
        model = Sequential()
        # model.add(Dense(100, input_dim = 2,  kernel_initializer='he_uniform', activation='relu'))
        model.add(Dense(100, input_dim = phi.shape[1],  
                kernel_initializer='he_uniform', activation='relu'))
        # model.add(Dropout(rate=0.5))
        # model.add(BatchNormalization())
        model.add(Dense(50, activation='relu'))
        model.add(Dense(50, activation='relu'))
        #     model.add(Dense(100, activation='relu'))
        # model.add(Dense(100, activation='relu'))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(10, activation='relu'))
        model.add(Dense(10, activation='relu'))
        #     model.add(Dense(50, activation='relu'))
        #     model.add(Dense(50, activation='relu'))
        #     model.add(Dense(10, activation='relu'))
        #model.add(Dense(50, activation='relu'))
        #model.add(Dropout(rate=0.5))
        #     model.add(Dense(10, activation='relu'))
        #model.add(BatchNormalization())
        model.add(Dense(10, activation='relu'))
        model.add(Dense(n, activation='softmax'))
        NB_START_EPOCHS = 50 
        # NB_START_EPOCHS = 200  # Number of epochs we usually start to train with
        BATCH_SIZE = 64  
        fold_no = 1
        optimizer = keras.optimizers.Adam(lr=0.001)
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

        callbacks = [EarlyStopping(monitor='val_accuracy', patience=200),
                     ModelCheckpoint(filepath='indicator_kriging.h5', 
                                     monitor='val_accuracy', save_best_only=True)]
        print("running model for threshold "+str(h+1)+".")
        result = model.fit(phi, dummy_y, callbacks=callbacks, class_weight = class_weight_dict,
                   validation_split = 0.1, epochs = 500, batch_size = 64, verbose = 0)

        model = keras.models.load_model('indicator_kriging.h5')
        return model
    model_list.append(model_function())

            x         y      var1      var2  proj_var1  proj_var2  threshold  \
509  0.910112  0.662921 -3.296573 -5.014667  -3.448834  -4.787949          1   

     dist_frm_orig  threshold2  threshold_var1  threshold_var2  
509      17.259161           4              26              16  
20
1167


 /Users/nagp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 /Users/nagp/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning:Pass classes=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19], y=509      3
27       2
661      5
1226    10
969     15
        ..
512      6
893      4
648      3
895      6
93      11
Name: threshold2, Length: 1167, dtype: int64 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error


running model for threshold 1.
            x    y      var1      var2  proj_var1  proj_var2  threshold  \
1291  0.11236  0.0 -3.845167 -4.546533  -3.781946  -4.631694          2   

      dist_frm_orig  threshold2  threshold_var1  threshold_var2  
1291      22.185913           5              23              20  
20
1119
running model for threshold 2.


 /Users/nagp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 /Users/nagp/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning:Pass classes=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19], y=1291     4
1292     3
1293     3
1294     3
1295     4
        ..
2405    18
2406    16
2407    18
2408     8
2409     8
Name: threshold2, Length: 1119, dtype: int64 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error


             x         y      var1      var2  proj_var1  proj_var2  threshold  \
2875  0.752809  0.449438 -4.470393 -4.792529  -4.436628   -4.83253          3   

      dist_frm_orig  threshold2  threshold_var1  threshold_var2  
2875      33.708485           7              20              17  
20
993


...

running model for threshold 5.
             x         y      var1      var2  proj_var1  proj_var2  threshold  \
6523  0.044944  0.550562 -6.097978 -5.206317  -6.006661  -5.304751          6   

      dist_frm_orig  threshold2  threshold_var1  threshold_var2  
6523       67.13107          12              10              15  
20
1042


 /Users/nagp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 /Users/nagp/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning:Pass classes=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19], y=6523    11
6648     5
5759    14
5843     5
6137     1
        ..
6109     1
6401     1
6573     3
6835    12
6824    11
Name: threshold2, Length: 1042, dtype: int64 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error


running model for threshold 6.
             x         y      var1      var2  proj_var1  proj_var2  threshold  \
7417  0.280899  0.393258 -3.156011 -2.183487  -3.121223  -2.227485          7   

      dist_frm_orig  threshold2  threshold_var1  threshold_var2  
7417      15.832108           3              27              31  
20
1093


 /Users/nagp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 /Users/nagp/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning:Pass classes=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19], y=7417     2
7939     8
6958     7
7811     2
7987    10
        ..
8004    14
7911     9
7161     7
7475     4
7172     2
Name: threshold2, Length: 1093, dtype: int64 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error


running model for threshold 7.


In [53]:
N = len(df_test)
s = np.vstack((df_test["x"],df_test["y"])).T

num_basis = [2**2,5**2,9**2]
knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
##Wendland kernel
K = 0
phi_test = np.zeros((N, sum(num_basis)))

for res in range(len(num_basis)):
    
    theta = 1/np.sqrt(num_basis[res])*2.5
    knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
    knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(s-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi_test[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi_test[j,i + K] = 0
    K = K + num_basis[res]
y_test = np.array(df_test[["var1","var2"]])




print(df_train.head(1))
y = np.array(df_train[["var1","var2"]])
N = len(df_train)
s = np.vstack((df_train["x"],df_train["y"])).T

num_basis = [2**2,5**2,9**2]
knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
##Wendland kernel
K = 0
phi = np.zeros((N, sum(num_basis)))

for res in range(len(num_basis)):
    theta = 1/np.sqrt(num_basis[res])*2.5
    knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
    knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(s-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi[j,i + K] = 0
    K = K + num_basis[res]

# DeepKriging model for continuous data
model = Sequential()
# model.add(Dense(100, input_dim = 2,  kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(100, input_dim = phi.shape[1],  
                kernel_initializer='he_uniform', activation='relu'))
# model.add(Dropout(rate=0.5))
# model.add(BatchNormalization())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
#     model.add(Dense(100, activation='relu'))
# model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
#     model.add(Dense(50, activation='relu'))
#     model.add(Dense(50, activation='relu'))
#     model.add(Dense(10, activation='relu'))
#model.add(Dense(50, activation='relu'))
#model.add(Dropout(rate=0.5))
#     model.add(Dense(10, activation='relu'))
#model.add(BatchNormalization())
model.add(Dense(10, activation='relu'))
model.add(Dense(2, activation='linear'))
#     quantile = 0.5
optimizer = keras.optimizers.Adam(lr=0.001)
model.compile(loss='mse', optimizer=optimizer)
#     model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = [EarlyStopping(monitor='val_loss', patience=200),
             ModelCheckpoint(filepath='mean_model.h5', monitor='val_loss', save_best_only=True)]
print("running model for mean. ")
result = model.fit(phi, y, callbacks=callbacks,
                   validation_data = (phi_test,y_test), epochs = 500, batch_size = 128, verbose = 0)

model_mean = model #keras.models.load_model('mean_model.h5')

             x         y      var1      var2  proj_var1  proj_var2  threshold  \
4560  0.157303  0.988764 -5.350081 -5.250944  -5.306286  -5.300444          4   

      dist_frm_orig  threshold2  threshold_var1  threshold_var2  
4560      50.197071          10              15              15  
running model for mean. 


In [54]:
pred = model_mean.predict(phi_test)
df_test["var1_mean"] = pred[:,0]
df_test["var2_mean"] = pred[:,1]

 /Users/nagp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 /Users/nagp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [55]:
a = np.array([-2.47176554,-1.82406163,-1.08755835,-0.60572381,0.03170641,0.26759939,0.24032180])
b = np.array([0.6715845,0.7423779,0.8441032,0.8847470,0.9492417,0.9276952,0.7906538])

df_test.reset_index()
for i in range(len(df_test)):
    vec_distance = abs(df_test.iloc[i]["var2_mean"] - b*df_test.iloc[i]["var1_mean"] - a)/np.sqrt(1+b**2)
    index = np.where(vec_distance == min(vec_distance))
    
#     print(index[0])
#     print(df_test.iloc[i]['threshold'])
    df_test.iloc[i]['threshold'] = index[0] + 1
#     print(df_test.iloc[i]['threshold'])

 /Users/nagp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [56]:
np.unique(df_test["threshold"])

array([1, 2, 3, 4, 5, 6, 7])

In [57]:
for h in range(7):
    df_test1 = df_test.loc[df_test['threshold'] == h+1]
    #     print(df_test1.head())
    N = len(df_test1)
    s = np.vstack((df_test1["x"],df_test1["y"])).T

    num_basis = [2**2,5**2,9**2]
    knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
    ##Wendland kernel
    K = 0
    phi = np.zeros((N, sum(num_basis)))

    for res in range(len(num_basis)):
        theta = 1/np.sqrt(num_basis[res])*2.5
        knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
        knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
        for i in range(num_basis[res]):
            d = np.linalg.norm(s-knots[i,:],axis=1)/theta
            for j in range(len(d)):
                if d[j] >= 0 and d[j] <= 1:
                    phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
                else:
                    phi[j,i + K] = 0
        K = K + num_basis[res]
    print(h)
    model = model_list[h]
    pred = model.predict(phi)
    df_test1 = df_test1.reset_index(drop=True)
    pred_df = pd.DataFrame(pred)
    pred_df = pred_df.reset_index(drop=True)
    df_test1 = pd.concat([df_test1,pred_df], axis = 1)
    if(h == 0):
        new_df_test = df_test1 
    else:
        print(h)
        new_df_test = pd.concat([new_df_test,df_test1], axis = 0)

0
1
1
2
2
3
3
4
4
5
5
6
6


In [58]:
new_df_test

,x,y,var1,var2,proj_var1,proj_var2,threshold,dist_frm_orig,threshold2,threshold_var1,...,10,11,12,13,14,15,16,17,18,19
0,0.022472,0.719101,-2.460593,-4.079591,-2.439918,-4.110376,1,8.638244,2,32,...,1.744112e-27,2.734075e-18,0.000000e+00,0.000000e+00,5.900360e-37,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,0.044944,0.797753,-3.056903,-4.681487,-3.129453,-4.573458,1,14.210588,3,28,...,3.184049e-17,5.206039e-16,5.226224e-28,9.211120e-36,7.902752e-33,3.198152e-36,9.362142e-35,0.000000e+00,0.000000e+00,0.000000e+00
2,0.000000,0.842697,-3.806602,-5.418866,-3.987406,-5.149646,1,23.070451,5,24,...,8.155365e-15,2.045845e-17,3.019527e-26,0.000000e+00,2.015293e-35,0.000000e+00,4.725469e-37,0.000000e+00,0.000000e+00,0.000000e+00
3,0.932584,0.707865,-3.029205,-4.408601,-2.984064,-4.475817,1,12.920859,3,29,...,6.194667e-25,3.153977e-15,1.588724e-36,0.000000e+00,3.462253e-35,0.000000e+00,3.775941e-37,0.000000e+00,0.000000e+00,0.000000e+00
4,0.943820,0.044944,-3.257951,-5.119515,-3.470744,-4.802663,1,17.479150,4,27,...,5.786769e-25,4.078878e-21,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,0.067416,0.460674,-5.744804,-4.230377,-5.710041,-4.274344,7,52.986778,12,13,...,8.023334e-01,5.853111e-02,1.053958e-07,3.221059e-07,4.484468e-17,0.000000e+00,2.747080e-16,7.394674e-34,2.014575e-30,0.000000e+00
136,0.584270,0.438202,-2.397881,-1.739886,-2.438901,-1.688005,7,9.666680,2,32,...,2.947227e-20,0.000000e+00,0.000000e+00,3.583898e-22,0.000000e+00,4.976700e-21,5.121621e-32,1.669491e-08,9.433476e-06,8.063034e-10
137,0.370787,0.438202,-3.288142,-2.057056,-3.141018,-2.243136,7,16.033554,3,27,...,1.262687e-15,3.577124e-38,0.000000e+00,2.956096e-24,0.000000e+00,0.000000e+00,5.661714e-38,4.081562e-21,7.661450e-16,9.460067e-20
138,0.393258,0.370787,-2.794351,-1.605171,-2.617321,-1.829073,7,11.132766,2,30,...,9.490100e-24,0.000000e+00,0.000000e+00,7.221601e-28,0.000000e+00,1.679113e-32,0.000000e+00,4.853921e-14,4.207175e-09,3.960791e-15


In [59]:
new_df_test.to_csv("median_results.csv")

In [60]:
df_train.to_csv("train_data.csv")